In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("generating input data") \
        .config("spark.sql.shuffle.partitions",3) \
        .master("local[2]") \
        .getOrCreate()

In [2]:
orders_schema = "order_id long, order_date string, customer_id long, order_status string"

In [11]:
orders_df = spark.read \
.format("csv") \
.option("header", "true") \
.schema(orders_schema) \
.load("../data/retail_db/orders/")

In [13]:
orders_df.createOrReplaceTempView("orders")

In [14]:
order_items_schema = "order_item_id long,order_item_order_id long,order_item_product_id long,order_item_quantity long,order_item_subtotal float,order_item_product_price float"

In [17]:
order_items_df = spark.read \
.format("csv") \
.option("header", "true") \
.schema(order_items_schema) \
.load("../data/retail_db/order_items/")

In [19]:
order_items_df.createOrReplaceTempView("order_items")

In [20]:
customers_schema = "customerid long,customer_fname string,customer_lname string,username string,password string,address string,city string,state string,pincode long"

In [26]:
customers_df =  spark.read \
.format("csv") \
.option("header", "true") \
.schema(customers_schema) \
.load("../data/retail_db/customers")

In [27]:
customers_df.createOrReplaceTempView("customers")

In [23]:
spark.sql("select * from orders limit 10").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+



In [24]:
spark.sql("select * from order_items limit 10").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [28]:
spark.sql("select * from customers limit 10").show()

+----------+--------------+--------------+---------+---------+--------------------+-----------+-----+-------+
|customerid|customer_fname|customer_lname| username| password|             address|       city|state|pincode|
+----------+--------------+--------------+---------+---------+--------------------+-----------+-----+-------+
|         1|       Richard|     Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|Brownsville|   TX|  78521|
|         2|          Mary|       Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|  Littleton|   CO|  80126|
|         3|           Ann|         Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|     Caguas|   PR|    725|
|         4|          Mary|         Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common| San Marcos|   CA|  92069|
|         5|        Robert|        Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|     Caguas|   PR|    725|
|         6|          Mary|         Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|    Passaic|   NJ|   7055|
|         

In [29]:
spark.sql("""SELECT order_id, order_item_id, customer_id, order_item_product_id, order_item_quantity, order_item_subtotal, order_item_product_price, customer_fname, customer_lname, city, state, pincode
FROM orders
JOIN order_items ON orders.order_id = order_items.order_item_order_id
JOIN customers ON orders.customer_id = customers.customerid""").show()

+--------+-------------+-----------+---------------------+-------------------+-------------------+------------------------+--------------+--------------+-----------+-----+-------+
|order_id|order_item_id|customer_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|customer_fname|customer_lname|       city|state|pincode|
+--------+-------------+-----------+---------------------+-------------------+-------------------+------------------------+--------------+--------------+-----------+-----+-------+
|       1|            1|      11599|                  957|                  1|             299.98|                  299.98|          Mary|        Malone|    Hickory|   NC|  28601|
|       2|            2|        256|                 1073|                  1|             199.99|                  199.99|         David|     Rodriguez|    Chicago|   IL|  60625|
|       2|            3|        256|                  502|                  5|              250.0|  

In [31]:
joined_df = spark.sql("""SELECT order_id, order_item_id, customer_id, order_item_product_id, order_item_quantity, order_item_subtotal, order_item_product_price, customer_fname, customer_lname, city, state, pincode
FROM orders
JOIN order_items ON orders.order_id = order_items.order_item_order_id
JOIN customers ON orders.customer_id = customers.customerid""")

In [35]:
from pyspark.sql.functions import *

In [36]:
result_df = joined_df.groupBy("order_id","customer_id","customer_fname","customer_lname","city","state","pincode").agg(collect_list(struct("order_item_id", "order_item_product_id","order_item_quantity","order_item_product_price","order_item_subtotal")).alias("line_items")).orderBy("order_id")

In [37]:
result_df.show(20,False)

+--------+-----------+--------------+--------------+-------------+-----+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|customer_id|customer_fname|customer_lname|city         |state|pincode|line_items                                                                                                                                           |
+--------+-----------+--------------+--------------+-------------+-----+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|1       |11599      |Mary          |Malone        |Hickory      |NC   |28601  |[{1, 957, 1, 299.98, 299.98}]                                                                                                                        |
|2       |256        |David         |Rodriguez     |Chicago      |IL   |6062

In [38]:
result_df \
.repartition(1) \
.write \
.format("json") \
.mode("overwrite") \
.option("path","../data/retail_db/data_json_orders") \
.save()

In [ ]:
spark.stop()